In [1]:
import os

In [2]:
%pwd

'/Users/omidsardari/WORK/Becoming a Data Scientist/Python Projects/End_to_End_Text_Summarizer_AWS/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/omidsardari/WORK/Becoming a Data Scientist/Python Projects/End_to_End_Text_Summarizer_AWS'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories
from textSummarizer.entity import (DataIngestionConfig, DataValidationConfig, DataTransformationConfig)


In [7]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])



    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.training_arguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        return model_trainer_config
    


In [8]:
import os
import torch
from textSummarizer.logging import logger
from textSummarizer.entity import ModelTrainerConfig
from transformers import AutoTokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk

/Users/omidsardari/anaconda3/envs/text_summarization/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # data loading
        dataset_samsum_pt = load_from_disk(self.config.data_path)


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                        tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                        train_dataset=dataset_samsum_pt["test"],
                        eval_dataset=dataset_samsum_pt["validation"]
        )

        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [9]:
import os
import torch
from textSummarizer.logging import logger
from textSummarizer.entity import ModelTrainerConfig
from transformers import AutoTokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM
from datasets import load_from_disk

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        model_dir = os.path.join(self.config.root_dir, "pegasus-samsum-model")
        tokenizer_dir = os.path.join(self.config.root_dir, "tokenizer")

        # Check if model and tokenizer already exist
        if os.path.isdir(model_dir) and os.listdir(model_dir) and os.path.isdir(tokenizer_dir) and os.listdir(tokenizer_dir):
            logger.info("Model and tokenizer already exist. Skipping training!")
            return

        device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
        os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # Data loading
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps, per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size, weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps, evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps, save_steps=1e6, gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model=model_pegasus, args=trainer_args, tokenizer=tokenizer, data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["test"], eval_dataset=dataset_samsum_pt["validation"]
        )

        trainer.train()

        # Save model
        model_pegasus.save_pretrained(model_dir)
        # Save tokenizer
        tokenizer.save_pretrained(tokenizer_dir)


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e    

[2023-10-26 18:08:36,291: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-26 18:08:36,293: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-26 18:08:36,294: INFO: common: created directory at: artifacts]
[2023-10-26 18:08:36,294: INFO: common: created directory at: artifacts/model_trainer]
[2023-10-26 18:08:36,295: INFO: 834295108: Model and tokenizer already exist. Skipping training.]


In [ ]:
#the lines of code to download trained pegasus model and tokenizer!

#from google.colab import files
#files.download('/content/pegasus-samsum-model')
#files.download('/content/tokenizer')